# Дополнительная лекция 1. Garbage collector в Python

Как вы помните из первой лекции, Python — это интерпретируемый язык программирования, поэтому перед запуском программы код на языке Python компилируется в машиночитаемые инструкции — байт-код. Инструкции байт-кода интерпретируются виртуальной машиной, определяемой реализацией языка, например, стандартной — CPython.

Оговоримся, что CPython не взаимодействует напрямую с регистрами и ячейками физической памяти — только с ее виртуальным представлением. В начале выполнения программы операционная система создает новый процесс и выделяет под него ресурсы. 

Выделенную виртуальную память интерпретатор использует для:
1. Собственной корректной работы;
2. Стека вызываемых функций и их аргументов;
3. Хранилища данных, представленного в виде кучи.

В отличие от C/C++, мы не можем управлять состоянием кучи напрямую из Python. Функции низкоуровневой работы с памятью предоставляются [Python/C API](https://docs.python.org/3/c-api/index.html), но обычно интерпретатор просто обращается к хранилищу данных через диспетчер памяти Python (memory manager). Очевидно, программа не сама выполняет сохранение и освобождение памяти — ведь мы не пишем соответствующих инструкций. Интерпретатор лишь запрашивает диспетчер памяти сделать это. А диспетчер уже делегирует работу, связанную с хранением данных, [аллокаторам](https://ru.wikipedia.org/wiki/Аллокатор) — распределителям памяти.

> Две программы не могут одновременно записать переменную в одно место виртуальной памяти, и отвечает за это [GIL (Global Interpreter Lock)](https://ru.wikipedia.org/wiki/Глобальная_блокировка_интерпретатора) — глобальная блокировка интерпретатора. GIL гарантирует: в один и тот же момент времени байт-код выполняется только одним потоком. Главное преимущество — безопасная работа с памятью, а основной недостаток в том, что многопоточное выполнение программ Python требует специфических решений.

# Организация доступной виртуальной памяти

Непосредственно с оперативной памятью взаимодействует распределитель сырой памяти (raw memory allocator). Поверх него работают аллокаторы, реализующие стратегии управления памятью, специфичные для отдельных типов объектов. Объекты разных типов — например, числа и строки — занимают разный объем, к ним применяются разные механизмы хранения и освобождения памяти. Аллокаторы стараются не занимать лишнюю память до тех пор, пока она не станет совершенно необходимой — этот момент определен стратегией распределения памяти CPython.

Обычно мы в своих программах не оперируем большими объектами. Большая часть наших данных — это числа, строки и т.п., они занимают не такой уж большой объем в расчёте на одно значение. Но зато мы создаем их достаточно часто. И это приводило бы к проблемам, если бы Python абсолютно все такие вызовы транслировал в операционную систему. Системный вызов на выделение памяти — штука трудозатратная, зачастую связанная с переходом в контекст ядра операционной системы и т.п. Поэтому одна из главных задач аллокатора Python — оптимизация количества системных вызовов.

Python использует динамическую стратегию, то есть распределение памяти выполняется во время выполнения программы. Виртуальная память Python представляет иерархическую структуру, оптимизированную под объекты Python размером менее 256 Кб:

* Арена — фрагмент памяти, расположенный в пределах непрерывного блока оперативной памяти объемом 256 Кб. Объекты размером более 256 Кб направляются в стандартный аллокатор C.
* Пул — блок памяти внутри арены, занимающий 4 Кб, что соответствует одной странице виртуальной памяти. То есть одна арена включает до 256/4 = 64 пулов.
* Блок — элемент пула размером от 16 до 512 байт. В пределах пула все блоки имеют одинаковый размер. Размер блока определяется тем, сколько байт требуется для представления конкретного объекта. Размеры блоков кратны 16 байт. То есть существует всего 512/16 = 32 классов (size class) блоков. То есть в одном пуле, в зависимости от класса, может находиться от 8 до 256 блоков.

![VirtualMemory](./pics/virt_mem_python.jpeg)
> Схематическое представление структуры виртуально памяти Python: арены состоят из пулов, пулы составлены из блоков

## Блок

Блок содержит не более одного объекта Python и находится в одном из трех состояний:

* `untouched` — блок еще не использовался для хранения данных;
* `free` — блок использовался механизмом памяти, но больше не содержит используемых программой данных;
* `allocated` — блок хранит данные, необходимые для выполнения программы.

В пределах пула блоки `free` организованы в односвязный список с указателем `freeblock`. Если аллокатору для выделения памяти не хватит блоков списка `freeblock`, он задействует блоки `untouched`. Освобождение памяти означает всего лишь то, что аллокатор меняет статус блока с `allocated` на `free` и начинает отслеживать блок в списке `freeblock`.

## Пул

Пул может находиться в одном из трех состояний: `used` (занят), `full` (заполнен), `empty` (пуст). Пустые пулы отличаются от занятых отсутствием блоков `allocated` и тем, что для них пока не определен size class. Пулы `full` полностью заполнены блоками `allocated` и недоступны для записи. Стоит освободиться любому из блоков заполненного пула — и он помечается как `used`.

Пулы одного типа и одного размера блоков организованы в двусвязные списки. Это позволяет алгоритму легко находить доступное пространство для блока заданного размера. Алгоритм проверяет список `usedpools` и размещает блок в доступном пуле. Если в `usedpools` нет ни одного подходящего пула для запроса, алгоритм использует пул из списка `freepools`, который отслеживает пулы в состоянии `empty`.

## Арена

Арены содержат пулы любых видов и организованы в двусвязный список `usable_arenas`. Список отсортирован по количеству доступных пустых пулов. Чем меньше в арене таких пулов, тем она ближе к началу списка. Для размещения новых данных выбирается область, наиболее заполненная данными.

Информацию о текущем распределении памяти в аренах, пулах и блоках можно посмотреть, запустив функцию `sys._debugmallocstats()`:

In [ ]:
import sys

sys._debugmallocstats()

Чтобы не произошло утечки памяти, диспетчер памяти должен отследить, что вся выделенная память освободится после завершения работы программы. То есть при завершении программы CPython дает задание очистить все арены.

Именно количество используемых арен определяет объем оперативной памяти, занимаемой программой на Python — если в арене все пулы в состоянии `empty`, СPython делает запрос на освобождение этого участка виртуальной памяти. Но уже понятно: чтобы пулы стали `empty`, все их блоки должны быть `free` или `untouched`. Получается, нужно понять, как CPython освобождает память.

# Алгоритмы сборки мусора

Стандартный интерпретатор питона (CPython) использует сразу два алгоритма, подсчет ссылок и generational garbage collector (далее GC), более известный как стандартный [модуль gc](https://docs.python.org/3.6/library/gc.html) из Python.

Алгоритм подсчета ссылок очень простой и эффективный, но у него есть один большой недостаток. Он не умеет определять циклические ссылки. Именно из-за этого, в питоне существует дополнительный сборщик, именуемый поколенческий GC, который следит за объектами с потенциальными циклическими ссылками.

В Python, алгоритм подсчета ссылок является фундаментальным и не может отключен, тогда как GC опционален и может быть отключен. 

## Алгоритм подсчета ссылок

Алгоритм подсчета ссылок это одна из самых простых техник для сборки мусора. Объекты удаляются как только на них больше нет ссылок.

В Python, переменные не хранят значения, а выступают в роли ссылок на объекты. То есть когда вы присваивайте значение новой переменной, то сначала создается объект с этим значением, а уже потом переменная начинает ссылаться на него. На один объект может ссылаться множество переменных.

Всё в Python является объектами, а прародителем всех типов объектов в реализации CPython является PyObject. От него наследуются все остальные типы. В PyObject определены счетчик ссылок и указатель на фактический тип объекта. Счетчик ссылок увеличивается на единицу, когда мы создаем что-то, что обращается к объекту, например, сохраняем объект в новой переменной. И наоборот, счетчик уменьшается на единицу, когда мы перестаем ссылаться на объект.

Примеры, когда количество ссылок увеличивается:

* оператор присваивания
* передача аргументов
* вставка нового объекта в list (увеличивается количество ссылок для объекта)
* конструция вида `foo = bar` (`foo` начинается ссылаться на тот же объект, что и `bar`)

Как только счетчик ссылок для определенного объекта достигает нуля интерпретатор запускает процесс уничтожения объекта. Если удаленный объект содержал ссылки на другие объекты, то эти ссылки также удаляются. Таким образом, удаление одного объекта может повлечь за собой удаление других. 

Например, если удаляется список, то счетчик ссылок во всех его элементах уменьшается на один. Если все объекты внутри списка больше нигде не используются, то они также будут удалены.

Переменные, которые объявлены вне функций, классов и блоков называются глобальными. Как правило, жизненный цикл таких переменных равен жизни Python процесса. Таким образом, количество ссылок на объекты на которые ссылаются глобальные переменные никогда не падает до нуля.

Переменные, которые объявлены внутри блока (функции, класса) имеют локальную видимость (т.е. они видны только внутри блока). Как только интерпретатор выходит из блока он уничтожает все ссылки созданные локальными переменными внутри него. 

Счетчик ссылок любого объекта можно проверить с помощью `sys.getrefcount()`. Учтите, что передача объекта в `getrefcount()` увеличивает счетчик ссылок на 1, так как сам вызов метода создает еще одну ссылку. Когда счетчик уменьшается до нуля, происходит вызов аллокатора для освобождения соответствующих блоков памяти.


In [ ]:
import sys

a = {}
b = a

print(sys.getrefcount(a))

del b

print(sys.getrefcount(a))

Однако счетчик ссылок неспособен отследить ситуации с циклическими ссылками. К примеру, возможна ситуация, когда два объекта ссылаются друг на друга, но оба уже не используются программой. Для борьбы с такими зависимостями используется сборщик мусора (garbage collector).

> Помимо невозможности для счетчика ссылок отслеживания циклических ссылок, он имеет так же проблемы связанные с блокированием потоков и дополнительные накладные расходы на память и процессор.

## Garbage collector

Если счетчик ссылок является свойством объекта, то сборщик мусора — механизм, который запускается на основе эвристик. Задача этих эвристик — снизить частоту и объем очищаемых данных. Основная стратегия заключается в разделении объектов на поколения: чем больше сборок мусора пережил объект, тем он значимее для выполнения работы программы. Сборщик мусора имеет интерфейс в виде [модуля gc](https://docs.python.org/3/library/gc.html).

Два примера циклических ссылок:

![cyclic](./pics/cyclic.svg)

Как вы можете видеть, объект `lst` ссылается сам на себя, тогда как `object1` и `object2` ссылаются друг на друга. Для таких объектов счетчик ссылок всегда будет равен 1.

Работает сборщик мусора следующим образом: GC отслеживает объекты (объекты-контейнеры, которые могут содержать ссылки на другие объекты) и смотрит, доступны ли они из основного кода на Python. Если нет, то сборщик их удаляет. Если да — оставляет.
В отличие от счетчика ссылок, механизм сборки мусора не работает постоянно. Он запускается от случая к случаю на основе эвристик. GC разделяет объекты на три поколения. Каждый новый объект начинает свой путь с первого поколения. Если объект переживает раунд сборки мусора, он переходит к более старому поколению. В младших поколениях сборка происходит чаще, чем в старших. Эта практика является стандартной для такого рода сборщиков мусора и снижает частоту и объем очищаемых данных. Идея простая: чем дольше живет объект, тем с большей вероятностью он проживет еще. То есть новые объекты удаляются гораздо чаще, чем те, которые уже просуществовали достаточно долго.
Каждое поколение имеет индивидуальный счётчик и порог. Счетчик хранит количество созданных минус количество удаленных объектов с момента последнего сбора. Каждый раз, когда вы создаете новый объект, Python проверяет, не превысил ли счетчик поколения пороговое значение. Если это так, Python инициирует процесс сборки мусора. 

Пример:

In [ ]:
import gc
import ctypes

# используется ctypes для доступа к объектам по адресу памяти
class PyObject(ctypes.Structure):
    _fields_ = [("refcnt", ctypes.c_long)]


gc.disable()  # выключаем циклический GC

lst = []
lst.append(lst)

# сохраняем адрес списка lst
lst_address = id(lst)

# удаляем ссылку lst
del lst

object_1 = {}
object_2 = {}
object_1['obj2'] = object_2
object_2['obj1'] = object_1

obj_address = id(object_1)

# удаляем ссылки
del object_1, object_2

# раскомментируйте для запуска ручной сборки объектов с циклическими ссылками
# gc.collect()

# проверяем счетчик ссылок
print(PyObject.from_address(obj_address).refcnt)
print(PyObject.from_address(lst_address).refcnt)